# EVA AI Hackathon — Submission Notebook

**Challenge:** `CH-08 — Training Effectiveness Analysis`

**Participant:** `Seif Eldin Mohamed`

**Date:** `2026-02-07`

**Runtime:** CPU

**Offline compliance:** No external APIs used.

---

## Submission Package
This notebook is part of your final submission ZIP:

# $1.$ **Importing Libraries**


In [1]:
# === 0) Setup: basic imports ===
import numpy as np
import pandas as pd
import urllib.request
import scipy.stats as stats
from pathlib import Path
import os


In [2]:
# === 0.1) Configuration (EDIT THESE) ===
CHALLENGE_CODE = "CH-08"
CHALLENGE_TITLE = "Measuring Training Effectiveness"
TEAM_NAME = "Seifaldeen Mohamed"



## 1) I/O Contract (Update when official schema is released)

### Input location
- `/data/`

### Output location
- `/output/`

### Expected input files
- `learner_event.csv` — Raw logs of course completions and exam scores.
- `kpis.csv` — Monthly employee performance data.

### Required output files (examples — replace with official list)
- `metrics_report.csv` — Detailed statistical output (Lift, P-value, Score Impact)
- `recomendation.csv` — Final strategic report with actionable business decisions

### Schema Reference
- Official schema source: **Data & Schema Addendum**
- This notebook must follow the official schema strictly (file names, columns, types, keys, constraints).


# $2.$ **Downloading and Loading Datasets**


In [3]:
# === 2) Inspect input folder ===
def load_data():
    path_1 = Path("data/learner_events.csv")
    path_2 = Path("data/kpis.csv")
    if not (path_1.is_file() and path_2.is_file()):
        Path("data").mkdir(parents=True, exist_ok=True)
        url1 = "https://github.com/Seifeldin6/training-effectiveness-analysis/raw/main/learner_events.csv"
        url2 = "https://github.com/Seifeldin6/training-effectiveness-analysis/raw/main/kpis.csv"
        urllib.request.urlretrieve(url1, path_1)
        urllib.request.urlretrieve(url2, path_2)
    return pd.read_csv(path_1), pd.read_csv(path_2)

df_events, df_kpi = load_data()

# $3.$ **Data Exploration**


In [4]:
df_events , df_events.info()

<class 'pandas.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   event_id        120000 non-null  str    
 1   ts              120000 non-null  str    
 2   learner_id      120000 non-null  str    
 3   module_id       120000 non-null  str    
 4   event_type      120000 non-null  str    
 5   score           59887 non-null   float64
 6   time_spent_min  120000 non-null  int64  
dtypes: float64(1), int64(1), str(5)
memory usage: 6.4 MB


(         event_id                   ts learner_id module_id   event_type  \
 0       LE0000001  2026-01-06T23:14:10    L004693     M0369     complete   
 1       LE0000002  2025-09-29T18:32:38    L007021     M0537  quiz_submit   
 2       LE0000003  2025-12-23T21:44:44    L007889     M0583      revisit   
 3       LE0000004  2025-11-29T12:29:42    L003262     M0561        start   
 4       LE0000005  2025-11-26T01:43:50    L001129     M0130     complete   
 ...           ...                  ...        ...       ...          ...   
 119995  LE0119996  2025-12-25T10:13:01    L008991     M0116     complete   
 119996  LE0119997  2025-10-27T11:08:02    L004769     M0180        start   
 119997  LE0119998  2025-11-23T00:49:39    L006165     M0256      revisit   
 119998  LE0119999  2025-12-06T14:54:51    L003081     M0460      dropoff   
 119999  LE0120000  2025-12-21T11:24:31    L004301     M0573     complete   
 
         score  time_spent_min  
 0         NaN              48  
 1      

In [5]:
df_events['ts'] = pd.to_datetime(df_events['ts'])
df_events.info()

<class 'pandas.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   event_id        120000 non-null  str           
 1   ts              120000 non-null  datetime64[us]
 2   learner_id      120000 non-null  str           
 3   module_id       120000 non-null  str           
 4   event_type      120000 non-null  str           
 5   score           59887 non-null   float64       
 6   time_spent_min  120000 non-null  int64         
dtypes: datetime64[us](1), float64(1), int64(1), str(4)
memory usage: 6.4 MB


In [6]:
df_kpi, df_kpi.info()

<class 'pandas.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   learner_id  100000 non-null  str    
 1   date        100000 non-null  str    
 2   kpi_value   99051 non-null   float64
dtypes: float64(1), str(2)
memory usage: 2.3 MB


(      learner_id        date  kpi_value
 0        L002002   8/11/2025      85.09
 1        L002530   9/14/2025      62.70
 2        L001475  12/15/2025      69.22
 3        L001542    4/4/2025      52.61
 4        L002642   9/23/2025      78.33
 ...          ...         ...        ...
 99995    L001856   7/21/2025      71.04
 99996    L002148   1/18/2025      61.61
 99997    L000602   6/10/2025      65.20
 99998    L000588   8/27/2025      79.14
 99999    L002010    5/8/2025      57.60
 
 [100000 rows x 3 columns],
 None)

In [7]:
df_kpi['date'] = pd.to_datetime(df_kpi['date'])
df_kpi.info()

<class 'pandas.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   learner_id  100000 non-null  str           
 1   date        100000 non-null  datetime64[us]
 2   kpi_value   99051 non-null   float64       
dtypes: datetime64[us](1), float64(1), str(1)
memory usage: 2.3 MB


In [8]:
df_kpi

,learner_id,date,kpi_value
0,L002002,2025-08-11,85.09
1,L002530,2025-09-14,62.70
2,L001475,2025-12-15,69.22
3,L001542,2025-04-04,52.61
4,L002642,2025-09-23,78.33
...,...,...,...
99995,L001856,2025-07-21,71.04
99996,L002148,2025-01-18,61.61
99997,L000602,2025-06-10,65.20
99998,L000588,2025-08-27,79.14


# $4.$ **Data Preprocessing**


In [9]:
df_completions = df_events[df_events['event_type'] == 'complete'].copy()

df_completions = df_completions.rename(columns={
    'ts': 'completion_date',
    'module_id': 'course_id'
})

if 'score' in df_completions.columns:
    df_completions['score'] = df_completions['score'].fillna(0)
else:
    df_completions['score'] = 0.0


df_train_clean = df_completions.sort_values(
    by=['learner_id', 'course_id', 'score'],
    ascending=[True, True, False]
)

df_train_clean = df_train_clean.drop_duplicates(
    subset=['learner_id', 'course_id'],
    keep='first'
)


df_train_clean = df_train_clean[['learner_id', 'course_id', 'completion_date', 'score']]

print(df_train_clean.head())

      learner_id course_id     completion_date  score
85017    L000001     M0043 2025-11-13 16:54:09   0.00
40627    L000001     M0131 2025-11-28 02:58:12   0.00
70264    L000001     M0284 2025-09-17 14:56:50   0.00
31759    L000001     M0320 2026-01-08 11:02:37  64.03
66245    L000001     M0420 2025-10-26 14:02:58   0.00


# $5.$ **Difference-in-Differences (DiD) Approximation**

In [10]:
merged_df = pd.merge(df_kpi, df_train_clean, on='learner_id', how='inner') # O(N+M)

merged_df['period'] = np.where(merged_df['date'] >= merged_df['completion_date'], 'Post', 'Pre')# O(R)

results = []

for course, group in merged_df.groupby('course_id'): # O(N)

    pre_values = group[group['period'] == 'Pre']['kpi_value'].dropna()
    post_values = group[group['period'] == 'Post']['kpi_value'].dropna()


    if len(pre_values) > 1 and len(post_values) > 1:


        avg_pre = pre_values.mean()
        avg_post = post_values.mean()
        lift = (avg_post - avg_pre) / avg_pre if avg_pre != 0 else 0.0

        t_stat, p_val = stats.ttest_ind(post_values, pre_values, equal_var=False)


        avg_score = df_train_clean[df_train_clean['course_id'] == course]['score'].mean()


        try:

            learner_lifts = []
            learner_scores = []
            for lid, l_group in group.groupby('learner_id'):
                l_pre = l_group[l_group['period']=='Pre']['kpi_value'].mean()
                l_post = l_group[l_group['period']=='Post']['kpi_value'].mean()
                if not np.isnan(l_pre) and not np.isnan(l_post) and l_pre!=0:
                    learner_lifts.append((l_post - l_pre)/l_pre)

                    s = l_group['score'].iloc[0]
                    learner_scores.append(s)

            if len(learner_scores) > 2 and np.std(learner_scores) > 0 and np.std(learner_lifts) > 0:
                corr, _ = stats.pearsonr(learner_scores, learner_lifts)
            else:
                corr = 0.0
        except:
            corr = 0.0

        results.append({
            'course_id': course,
            'lift': round(lift, 4),
            'p_value': round(p_val, 4),
            'avg_score': round(avg_score, 1),
            'score_impact': round(corr, 4) # (1 = Direct relationship, 0 = No relationship , -1 = Inverse relationship)
        })

df_results = pd.DataFrame(results)

print(df_results.head())

  course_id    lift  p_value  avg_score  score_impact
0     M0001  0.0970   0.0000       37.4        0.2737
1     M0002  0.0628   0.0012       42.1        0.0663
2     M0003  0.1432   0.0000       33.9       -0.0844
3     M0004  0.1541   0.0000       39.0       -0.3313
4     M0005  0.1374   0.0000       27.7        0.2069


### **$6.$ Output report**

In [11]:
def generate_detailed_insights(row):
    if row['p_value'] >= 0.05:
        return "Inconclusive Results (No Significant Change)"

    else:
        if row['lift'] > 0:
            if row['score_impact'] > 0.1:
                return "Ideal Success"
            elif row['score_impact'] < -0.1:
                return "Exam Mismatch"
            else:
                return "General Improvement"

        else:
            return "Significant Negative"

if not df_results.empty:

    df_results['insights'] = df_results.apply(generate_detailed_insights, axis=1)


    OUTPUT_DIR = 'output'
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    output_path = f"{OUTPUT_DIR}/metrics_report.csv"
    df_results.to_csv(output_path, index=False)

    print("\nInsight Categories Distribution:")
    print(df_results['insights'].value_counts())

    print("\n--- Preview of Critical Cases (Exam Mismatch) ---")
    print(df_results[df_results['insights'].str.contains('Mismatch')][['course_id', 'lift', 'score_impact']].head())

else:
    print("No valid results to save.")


Insight Categories Distribution:
insights
Exam Mismatch                                   220
Ideal Success                                   205
General Improvement                             153
Inconclusive Results (No Significant Change)     22
Name: count, dtype: int64

--- Preview of Critical Cases (Exam Mismatch) ---
   course_id    lift  score_impact
3      M0004  0.1541       -0.3313
7      M0008  0.1108       -0.1192
10     M0011  0.1436       -0.1594
11     M0012  0.1827       -0.1912
15     M0016  0.1359       -0.5961


### **$7.$ Output recommendation**

In [12]:
def get_action_plan(row):
    category = row['insights']

    if "Ideal Success" in category:
        meaning = "High exam scores correlate with high job performance."
        action = "Scale to more teams; Use exam for promotion; Reward high scorers."

    elif "Exam Mismatch" in category:
        meaning = "Course is effective, BUT exam scores are misleading (Negative Correlation)."
        action = "URGENT: Stop using current exam for evaluation; Replace with practical project."

    elif "General Improvement" in category:
        meaning = "Performance improved for everyone regardless of exam score."
        action = "Keep as knowledge sharing; Simplify or remove the exam to save costs."

    elif "Significant Negative" in category:
        meaning = "Performance declined after training."
        action = "STOP course immediately; Review content and target audience."

    else: # Inconclusive
        meaning = "No statistically significant change detected."
        action = "Collect more data; Re-evaluate in next quarter."

    return pd.Series([meaning, action], index=['meaning', 'recommendation'])

if not df_results.empty:


    action_details = df_results.apply(get_action_plan, axis=1)
    df_final = pd.concat([df_results, action_details], axis=1)

    output_cols = [
        'course_id',
        'lift',
        'p_value',
        'avg_score',
        'score_impact',
        'insights',
        'meaning',
        'recommendation'
    ]

    df_final = df_final[output_cols]

    OUTPUT_DIR = 'output'
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    output_path = f"{OUTPUT_DIR}/recommendation.csv"
    df_final.to_csv(output_path, index=False)


    print("\n--- Recommendation File Preview ---")
    print(df_final[['course_id', 'insights', 'recommendation']].head())

else:
    print("No results to process.")


--- Recommendation File Preview ---
  course_id             insights  \
0     M0001        Ideal Success   
1     M0002  General Improvement   
2     M0003  General Improvement   
3     M0004        Exam Mismatch   
4     M0005        Ideal Success   

                                      recommendation  
0  Scale to more teams; Use exam for promotion; R...  
1  Keep as knowledge sharing; Simplify or remove ...  
2  Keep as knowledge sharing; Simplify or remove ...  
3  URGENT: Stop using current exam for evaluation...  
4  Scale to more teams; Use exam for promotion; R...  
